# **ETL datos de YELP**
---


In [2]:
# Importacion de librerias a utilizar
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle
import pyarrow.parquet as pq

# Establece conexión con google Drive
drive.mount('/content/drive')

def aplicar_ETL():
  # Variables donde se almacenan las rutas de los archivos crudos
  ruta1='/content/drive/MyDrive/Proyecto Final/Yelp/business.pkl'
  ruta2='/content/drive/MyDrive/Proyecto Final/Yelp/checkin.json'
  ruta3='/content/drive/MyDrive/Proyecto Final/Yelp/review.json'
  # ruta4='/content/drive/MyDrive/Proyecto Final/Yelp/tip.json'
  ruta5='/content/drive/MyDrive/Proyecto Final/Yelp/user.parquet'

  rutas=[ruta1,ruta2,ruta3,ruta5]

  for ruta in rutas:
    nombre_archivo=ruta.split('/')[-1].split('.')[0]


    # Desarrollo de ETL---------------------------------
    if nombre_archivo=='business':
      try:
        #  # Lee archivo y devuelve un dataframe de pandas
        business=pd.read_pickle(ruta)
        business=business.iloc[:,:14]
        # Seleccion de columna importantes
        df_business=business.drop(columns=['is_open','attributes','hours'])
        # Elimina valores nulos
        df_business=df_business.dropna().reset_index(drop=True)
        # Cambio de Tipos
        df_business['latitude']=df_business['latitude'].astype('float')
        df_business['longitude']=df_business['longitude'].astype('float')
        df_business['stars']=df_business['stars'].astype('int')
        df_business['review_count']=df_business['review_count'].astype('int')
        # Lista de estados seleccionados
        estados_seleccionados=['PA','FL','CA','IL']
        # Filtado registros por estados
        df_business=df_business[df_business['state'].isin(estados_seleccionados)].reset_index(drop=True)
        # Trabajaremos con los negocios que contengan mas de 20 reseñas, ya que menos reseñas no seria eficiente para el sistema de recomendación
        df_business=df_business[df_business['review_count']>20].reset_index(drop=True)
        # Filtrado de negocios por rubro gastronómico
        palabras_clave=["Restaurants","Food","Bubble Tea","Coffee & Tea","Bakery","Barbecue","Buffets","Burgers","Cafes","Catering","Cocktail Bars","Desserts","Diners","Food Trucks","Ice Cream & Frozen Yogurt","Juice Bars & Smoothies","Pizza","Seafood","Steakhouses","Sushi Bars","Tapas Bars","Wine Bars","Vegetarian","Vegan","Gluten-Free"]
        filtro_gastronomico=df_business['categories'].str.contains('|'.join(palabras_clave),case=False)
        df_business=df_business[filtro_gastronomico].reset_index(drop=True)
        # Exportar archivo en formato parquet
        df_business.to_parquet('business_procesado.parquet')
      except: pass


    elif nombre_archivo=='checkin':
      try:
        # Lee archivo y devuelve un dataframe de pandas
        checkin=pd.read_json(ruta,lines=True,nrows=5000)
        # Transforma los datos de la columna "date" en una lista de fechas
        checkin['date']=checkin['date'].str.split(',')
        # Expandir datos
        checkin=checkin.explode('date').reset_index(drop=True)
        # Cruzar con datos de df_business y filtrar unicamente los negocios del rubro gastronomico
        df_merge_checkin_business=pd.merge(checkin,df_business,on='business_id',how='inner')
        df_checkin=df_merge_checkin_business[checkin.columns]
        # Exportar archivo en formato parquet
        df_checkin.to_parquet('checkin_procesado.parquet')
      except: pass


    elif nombre_archivo=='review':
      try:
        # Lee archivo y devuelve un dataframe de pandas
        review=pd.read_json(ruta,lines=True,nrows=5000)
        # Cambiar nombre de variable "stars" para evitar errores al momento de cruzar los datos
        review.rename({'stars':'stars_review'},axis=1,inplace=True)
        # Cruzar con datos del data set con df_biusiness (resultado luego del etl del dataset "business")
        df_review=pd.merge(review,df_business,on='business_id',how='inner')
        df_review=df_review[review.columns]
        # Exportar archivo en formato parquet
        df_review.to_parquet('review_procesado.parquet')
      except: pass


    elif nombre_archivo=='user':
      try:
        tabla_parquet=pq.read_table(ruta5)
        user=tabla_parquet.to_pandas()
        # Lee archivo y devuelve un dataframe de pandas
        # user=pd.read_parquet(ruta5)
        #Eliminar registros duplicados
        df_user=user.drop_duplicates().reset_index(drop=True)
        # Filtrar usuarios con mayor cantidad de reseñas efectuadas (usuarios mas activos en la plataforma es un indicador de fiabilidad del comentario tanto positivo como negativo)
        df_user=df_user[df_user['review_count']>20]
        # Exportar archivo en formato parquet
        df_user.to_parquet('user_procesado.parquet')
      except:pass


  # # Muestra datasets ya procesados
  print('\n---------------------------------------------------------------------Dataset "business" procesado-------------------------------------------------------\n')
  display(df_business)
  print('\n---------------------------------------------------------------------Dataset "checkin" procesado--------------------------------------------------------\n')
  display(df_checkin)
  print('\n---------------------------------------------------------------------Dataset "review" procesado----------------------------------------------------------\n')
  display(df_review)
  print('\n---------------------------------------------------------------------Dataset "user" procesado----------------------------------------------------------\n')
  display(df_user)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
aplicar_ETL()